<a href="https://colab.research.google.com/github/nawnie/EveryDream2trainer/blob/main/Train_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EveryDream2 Colab Edition

Check out documentation here: https://github.com/victorchall/EveryDream2trainer#docs

And join the discord: https://discord.gg/uheqxU6sXN

In [ ]:
#@title # Install python 3.10 
#@markdown # This will show a runtime error, its ok, its on purpose to restart the kernel to update python.
import os
from IPython.display import clear_output
!wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
!bash ./py310.sh -b -f -p /usr/local
!python -m ipykernel install --name "py310" --user
clear_output()
os.kill(os.getpid(), 9)

In [ ]:
#@title Verify python version, should be 3.10.something
!python --version

In [ ]:
#@title Optional connect Gdrive
#@markdown # but strongly recommended
#@markdown This will let you put all your training data and checkpoints directly on your drive.  Much faster/easier to continue later, less setup time.

#@markdown Creates /content/drive/MyDrive/everydreamlogs/ckpt
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/everydreamlogs/ckpt

In [ ]:
#@markdown # Install Dependencies
#@markdown This will take a couple minutes, be patient and watch the output for "DONE!"
from IPython.display import clear_output
from subprocess import getoutput
s = getoutput('nvidia-smi')
!pip install -q torch==1.12.1+cu116 torchvision==0.13.1+cu116 --extra-index-url "https://download.pytorch.org/whl/cu116"
!pip install -q transformers==4.25.1
!pip install -q diffusers[torch]==0.10.2
!pip install -q pynvml==11.4.1
!pip install -q bitsandbytes==0.35.0
!pip install -q ftfy==6.1.1
!pip install -q aiohttp==3.8.3
!pip install -q tensorboard>=2.11.0
!pip install -q protobuf==3.20.1
!pip install -q wandb==0.13.6
!pip install -q pyre-extensions==0.0.23
if "A100" in s:
  !pip install -q https://huggingface.co/industriaditat/xformers_precompiles/blob/main/A100_13dev/xformers-0.0.13.dev0-py3-none-any.whl
else:
  !pip install -q https://huggingface.co/industriaditat/xformers_precompiles/resolve/main/T4_13dev/xformers-0.0.13.dev0-py3-none-any.whl
!pip install -q pytorch-lightning==1.6.5
!pip install -q OmegaConf==2.2.3
!pip install -q numpy==1.23.5
!pip install -q colorama
!pip install -q keyboard
clear_output()
!git clone https://github.com/victorchall/EveryDream2trainer.git
%cd /content/EveryDream2trainer
!wget "https://raw.githubusercontent.com/nawnie/EveryDream2trainer/main/train_colab.py"
clear_output()
print("DONE!")

In [ ]:
#@title Get A Base Model
#@markdown Choose SD1.5 or Waifu Diffusion 1.3 from the dropdown, or paste your own URL in the box

#@markdown If you already did this once with Gdrive connected, you can skip this step as the cached copy is on your gdrive
from IPython.display import clear_output
!mkdir input
%cd /content/EveryDream2trainer
!python utils/get_yamls.py
MODEL_URL = "https://huggingface.co/panopstor/EveryDream/resolve/main/sd_v1-5_vae.ckpt" #@param ["https://huggingface.co/panopstor/EveryDream/resolve/main/sd_v1-5_vae.ckpt", "https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float16.ckpt"] {allow-input: true}
print("Downloading ")
!wget $MODEL_URL

%cd /content/EveryDream2trainer

clear_output()
print("DONE!")

In order to train, you need a base model on which to train. This is a one-time setup to configure base models when you want to use a particular base.  


In [ ]:
import os
#@title Setup conversion

#@markdown **If you already did this once with Gdrive connected, you can skip this step as the cached copy is on your gdrive.**  
# 
# If you are not sure, look in your Gdrive for `everydreamlogs/ckpt` and see if you have a folder with the `save_name` below.

#@markdown Pick the `model_type` in the dropdown. This is the model type that you are converting and you downloaded above.  This is important as it will determine the model architecture and the correct settings to use.

#@markdown * `SD1x` is all SD1.x based models *(SD1.4, SD1.5, Waifu Diffusion 1.3, etc)*

#@markdown * `SD2_512_base` is the SD2 512 base model

#@markdown * `SD21` is all SD2 768 models.  *(ex. SD2.1 768, or trained models based on that)*

#@markdown If you are not sure, double check the model author's page or ask for help on [Discord](https://discord.gg/uheqxU6sXN).
model_type = "SD1x" #@param ["SD1x", "SD2_512_base", "SD21"]

#@markdown This is the temporary ckpt file that was downloaded above. If you downloaded a different model, you can change this.  *Hint: look at your file manager in the EveryDream2trainer folder for .ckpt files*.
base_path = "/content/EveryDream2trainer/sd_v1-5_vae.ckpt" #@param {type:"string"}

#@markdown The name that you will use when selecting this model in the future training sessons.
save_name = "SD15" #@param{type:"string"}

#@markdown If you are using Gdrive, this will save the converted model to your Gdrive for future use so you can skip downloading and converting the model.
cache_to_gdrive = True #@param{type:"boolean"}

if cache_to_gdrive:
    save_name = os.path.join("/content/drive/MyDrive/everydreamlogs/ckpt", save_name)

img_size = 512
upscale_attention = False
if model_type == "SD1x":
    inference_yaml = "v1-inference.yaml"
elif model_type == "SD2_512_base":
    upscale_attention = True
    inference_yaml = "v2-inference.yaml"
elif model_type == "SD21":
    upscale_attention = True
    inference_yaml = "v2-inference-v.yaml"
    img_size = 768

print(base_path)
print(inference_yaml)

!python utils/convert_original_stable_diffusion_to_diffusers.py --scheduler_type ddim  \
--original_config_file {inference_yaml} \
--image_size {img_size} \
--checkpoint_path {base_path} \
--prediction_type epsilon \
--upcast_attn False \
--dump_path {save_name}

In [ ]:
#@title Pick your base model from a diffusers model saved to your Gdrive (converted above)

#@markdown Do not skip this cell.

#@markdown * If you have preveiously saved diffusers on your drive you can select it here

#@markdown ex. */content/drive/MyDrive/everydreamlogs/myproject_202208/ckpts/interrupted-gs023*

#@markdown The default for SD1.5 converted above would be */content/drive/MyDrive/everydreamlogs/ckpt/SD15*
Resume_Model = "/content/drive/MyDrive/everydreamlogs/ckpt/SD15" #@param{type:"string"} 
save_name = Resume_Model

For a more indepth Explanation of each of these paramaters check out /content/EveryDream2trainer/doc.


After youve tried a few models you will find /content/EveryDream2trainer/doc/ATWEAKING.md to be extremly helpful.

In [ ]:
#@title 
#@markdown # Run Everydream 2
#@markdown If you want to use a .json config or upload your own, skip this cell and run the cell below instead

#@markdown * Save logs and output ckpts to Gdrive (strongly suggested)
Save_to_Gdrive = True #@param{type:"boolean"}
#@markdown * Use resume to contnue training you just ran, will also find latest diffusers log in your Gdrive to continue.
resume = False #@param{type:"boolean"}
#@markdown * Checkpointing Saves Vram to allow larger batch sizes minor slow down on a single batch size but will can allow room for a higher traning resolution (suggested on Colab Free tier, turn off for A100)
Gradient_checkpointing = True #@param{type:"boolean"}
Disable_Xformers = False
#@markdown * Tag shuffling, mainly for booru training. Best to just read this if interested in shufflng tags /content/EveryDream2trainer/doc/SHUFFLING_TAGS.md
shuffle_tags = False #@param{type:"boolean"}
#@markdown * You can turn off the text encoder training (generally not suggested)
Disable_text_Encoder= False #@param{type:"boolean"}
#@markdown * Name your project so you can find it in your logs
Project_Name = "my_project" #@param{type: 'string'}

#@markdown * The learning rate affects how much "training" is done on the model per training step. It is a very careful balance to select a value that will learn your data and not wreck the model. 
#@markdown Leave this default unless you are very comfortable with training and know what you are doing.

Learning_Rate = 1e-6 #@param{type: 'number'}

#@markdown * A learning rate scheduler can change your learning rate as training progresses.

#@markdown I recommend sticking with constant until you are comfortable with general training. 

Schedule = "constant" #@param ["constant", "polynomial", "linear", "cosine"] {allow-input: true}

#@markdown * Resolution to train at (recommend 512).  Higher resolution will require lower batch size (below).
Resolution = 512 #@param {type:"slider", min:256, max:768, step:64}

#@markdown * Batch size is also another "hyperparameter" of itself and there are tradeoffs. It may not always be best to use the highest batch size possible. Once of the primary reasons to change it is if you get "CUDA out of memory" errors where lowering the value may help.

#@markdown * Batch size impacts VRAM use.  8 should work on SD1.x models and 5 for SD2.x models at 512 resolution.  Lower this if you get CUDA out of memory errors. You can check resources on your instance and watch the GPU RAM.

Batch_Size = 6 #@param{type: 'number'}

#@markdown * Gradient accumulation is sort of like a virtual batch size increase use this to increase batch size with out increasing vram usage
#@markdown Increasing from 1 to 2 will have a minor impact on vram use, but more beyond that will not.
#@markdown In colab free teir you can expect the fastest proformance from a batch of 8 and a gradient step of 1
#@markdown This is mostly for use if you are training higher resolution on free tier and cannot increase batch size.

Gradient_steps = 1 #@param{type:"slider", min:1, max:10, step:1}

#@markdown * Location on your Gdrive where your training images are.
Dataset_Location = "/content/drive/MyDrive/training_samples" #@param {type:"string"}
dataset = Dataset_Location
model = save_name

#@markdown * Max Epochs to train for, this defines how many total times all your training data is used. Default of 100 is a good start if you are training ~30-40 images of one subject. If you have 100 images, you can reduce this to 40-50 and so forth.

Max_Epochs = 100 #@param {type:"slider", min:0, max:200, step:5}

#@markdown * How often to save checkpoints.
Save_every_N_epoch = 20 #@param{type:"integer"}

#@markdown * Test sample generation steps, how often to generate samples during training.

#@markdown You can set your own sample prompts by adding them, one line at a time, to `/content/EveryDream2trainer/sample_prompts.txt`.  If left empty, it will use the captions from your training images.

#@markdown Use the steps_between_samples to set how often the samples are generated.
Steps_between_samples = 300 #@param{type:"integer"}

#@markdown *Weights and Biases logging token. 

# #@markdown Paste your token here if you have an account so you can use it to track your training progress.  If you don't have an account, you can create one for free at https://wandb.ai/site.  Log will use your project name from above. This is a free online logging utility.

# #@markdown Your key is on this page: https://wandb.ai/settings under "Danger Zone" "API Keys"
wandb_token = '' #@param{type:"string"}

#@markdown * That's it!  Run the cell!

wandb_settings = ""
if wandb_token:
  !wandb login $wandb_token
  wandb_settings = "--wandb"

Drive=""
if Save_to_Gdrive:
  Drive = "--logdir /content/drive/MyDrive/everydreamlogs --save_ckpt_dir /content/drive/MyDrive/everydreamlogs/ckpt"

if Max_Epochs==0:
  Max_Epoch=1

if resume:
  model = "findlast"

Gradient = ""
if Gradient_checkpointing:
  Gradient = "--gradient_checkpointing "
if "A100" in s:
  Gradient = ""

DX = ""  
if Disable_Xformers:
  DX = "--disable_xformers "

shuffle = ""
if shuffle_tags:
  shuffle = "--shuffle_tags "

textencode = ""
if Disable_text_Encoder:
  textencode = "--disable_textenc_training Train_text "

!python train.py --resume_ckpt "$model" \
  $textencode \
  $Gradient \
  $shuffle \
  $Drive \
  $DX \
  $wandb_settings \
  --amp \
  --batch_size $Batch_Size \
  --grad_accum $Gradient_steps \
  --cond_dropout 0.00 \
  --data_root "$dataset" \
  --flip_p 0.00 \
  --lr $Learning_Rate \
  --lr_decay_steps 0 \
  --lr_scheduler "$Schedule" \
  --lr_warmup_steps 0 \
  --max_epochs $Max_Epochs \
  --project_name "$Project_Name" \
  --resolution $Resolution \
  --sample_prompts "sample_prompts.txt" \
  --sample_steps $Steps_between_samples \
  --save_every_n_epoch $Save_every_N_epoch \
  --seed 555 \
  --shuffle_tags \
  --useadam8bit 


In [ ]:
#@title Alternate startup script
#@markdown * Edit train.json to setup your paramaters
#@markdown * Edit chain0.json to make use of chaining
#@markdown * make sure to check each confguration you will need 1 Json per chain length 3 are provided
#@markdown * make sure your .Json contain the line Notebook: true


%cd /content/EveryDream2trainer
Chain_Length=0 #@param{type:"integer"}
l = Chain_Length 
I=0 #repeat counter
if l == None or l == 0:
  l=1
while l > 0:
  !python train_colab.py --config chain{I}.json
  l -= 1
  I =+ 1

In [ ]:
#@title Test your Diffusers
#@markdown Path to the diffusers that was trained

#@markdown You can look in the file drawer on the left /content/drive/MyDrive/everydreamlogs and click the three dots to copy the path

#@markdown ex. /content/drive/MyDrive/everydreamlogs/my_project_20230126-023804/ckpts/interrupted-gs86

diffusers_path="" #@param{type:"string"}
DF=diffusers_path
PROMPT= "a photo of an astronaut on the moon"#@param{type:"string"}
Resolution = 512 #@param {type:"slider", min:256, max:1024, step:32}
Seed= -1 #@param{type:"integer"}
Steps = 30 #@param {type:"slider", min:10, max:50, step:1}
cfg = 7 #@param {type:"slider", min:1, max:15, step:0.5}


!python /content/EveryDream2trainer/scripts/txt2img.py   \
 --diffusers_path "$DF" \
 --resolution $Resolution \
 --seed $Seed \
 --prompt "$PROMPT" \
 --steps $Steps \
 --cfg_scale $cfg 